In [1]:
import operator
import functools
import random

def foldl(func, acc, xs):
    return functools.reduce(func, xs, acc)  


In [2]:
foldl(operator.add, 0, [1,2,3,4,5,6,7,8,9,10])


55

In [3]:
# parity check, function = xor, acc = 0 sequece, 0s and 1s
random.seed(1)

print("   Bitstirng     |      Parity   ")
print("-"*34)

#generating a random 12 digit binary string

for _ in range(1):
    seq = [random.randint(0,1) for _ in range(12)]
    print(f"{''.join(str(b) for b in seq)}  |    {foldl(operator.xor, 0, seq)}")




   Bitstirng     |      Parity   
----------------------------------
001011110010  |    0


In [4]:
random.seed(1)

def traceXOR(a,b):
    """
    shows the intermediate steps of 
    xor function on 
    a sequence
    """

    result = operator.xor(a, b)
    print(f"{a} XOR {b} = {result}")
    return result

print(foldl(traceXOR, 0, [1, 0, 0, 1, 1]))

0 XOR 1 = 1
1 XOR 0 = 1
1 XOR 0 = 1
1 XOR 1 = 0
0 XOR 1 = 1
1


In [5]:
""" The math behind the above XOR operations is '0' is the initial accumulator value
so 0 XOR 1 == 1
now the accumulator value has been changed to 1 
so the second operation
1 XOR 0 = 1
1 XOR 0 = 1
1 XOR 1 = 0
0 XOR 1 = 1
1----> is the final accumulator value 
"""


" The math behind the above XOR operations is '0' is the initial accumulator value\nso 0 XOR 1 == 1\nnow the accumulator value has been changed to 1 \nso the second operation\n1 XOR 0 = 1\n1 XOR 0 = 1\n1 XOR 1 = 0\n0 XOR 1 = 1\n1----> is the final accumulator value \n"

In [6]:
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
#DATA
Training_Size = 100000
Validation_Size = 10000
Bit_len = 50
Variable_Len = True

# Model Parameters
InputSize = 1
HiddenSize = 2
NumberLayers = 1

#Training Parameters
BatchSize = 8
Epochs = 8
LearningRate = 0.01  #Default ADAM = 0.001
Threshold = 0.0001

In [33]:
class XOR(data.Dataset):
    def __init__(self,sample_size = Validation_Size, bit_len = Bit_len, variable=False):
        self.bit_len = Bit_len
        self.sample_size = Validation_Size
        self.variable = Variable_Len
        self.features, self.labels = self.generate_data(sample_size,bit_len)


    def __getitem__(self, index):
        return self.features[index, :], self.labels[index]
    
    def __len__(self):
        return len(self.features)

    def generate_data(self,sample_size, seq_length = Bit_len):
        bits = torch.randint(2, size=(sample_size, seq_length, 1)).float()
        if self.variable:
            # we generate random integers and pad the bits with zeros
            # to mimic variable bit string lengths 
            # padding with zeros as they do not provide information
           
            # pad = torch.randint(seq_length, size=(sample_size, ))

            # for idx, p in enumerate(pad):
            #     bits[idx, p:] = 0.

             # TODO: vectorize instead of loop?

            # Generate random integers for padding positions
            pad = torch.randint(seq_length, size=(sample_size, 1))
            """
            # Create a mask for zero-padding
            mask = torch.arange(seq_length).expand(sample_size, seq_length) >= pad

            # Apply the mask to set values after the padding positions to 0
            bits = bits * mask.float()

            bitsum = bits.cumsum(axis=1)
            """
            # if bitsum[i] odd: -> True
            # else: False
            for idx, p in enumerate(pad):
                bits[idx, p:] = 0.

        bitsum = bits.cumsum(axis=1)
        parity = (bitsum % 2 != 0).float()
            
        return bits, parity




In [34]:
# sample_size = 10
# seq_length = 2
# bits = torch.randint(2, size=(sample_size, seq_length, 1)).float()
# print(bits)

In [35]:
class XORLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, number_layers):
        super(XORLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.number_layers  = number_layers

        self.lstm = nn.LSTM(input_size, hidden_size, number_layers, batch_first=True)
        self.fc  = nn.Linear(hidden_size, 1)
        self.activation = nn.Sigmoid()


    def forward(self, x, lengths=True):
        h0 = torch.zeros(self.number_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.number_layers, x.size(0), self.hidden_size).to(device)
        out_lstm, _ = self.lstm(x,(h0,c0))
        out = self.fc(out_lstm)
        predictions = self.activation(out)
        return predictions
    



In [36]:
model = XORLSTM(InputSize, HiddenSize, NumberLayers).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LearningRate)

In [52]:
def train():
    model.train()
    train_loader = DataLoader(XOR(Training_Size,Bit_len,Variable_Len), batch_size=BatchSize)
    total_step = len(train_loader)

    print("Training...\n")
    print('-'*60)

    for epoch in range(1, Epochs+1):
        for step, (features, labels) in enumerate(train_loader):
            features, labels= features.to(device), labels.to(device)
            #forward pass
            output = model(features)
            loss = criterion(output, labels)

            #backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            accuracy = ((output>0.5)==(labels>0.5)).type(torch.FloatTensor).mean()

            if (step+1) % 250==0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.3f}' 
                .format(epoch, Epochs, step+1, total_step, loss.item(), accuracy))
                print('-'*60)

                if abs(accuracy-1.0)<Threshold:
                    print("Early Stopping")
                    return

            if (step+1)==total_step:
                valid_accuracy = validate(model)
                print("validation accuracy: {:.4f}".format(valid_accuracy))
                print('-'*60)
                if abs(valid_accuracy - 1.0) < THRESHOLD:
                    print("EARLY STOPPING")
                    return 


In [53]:
def validate(model):
    valid_loader = DataLoader(
                    XOR(Validation_Size, Bit_len, Variable_Len), 
                        batch_size=BatchSize
                        )
    model.eval()
    correct = 0.
    total = 0.
    for features, labels in valid_loader:
        features, labels = features.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(features)
            total += labels.size(0)*labels.size(1)
            correct += ((outputs > 0.5) == (labels > 0.5)).sum().item()
    return correct / total

In [54]:
train()

Training...

------------------------------------------------------------
Epoch [1/8], Step [250/12500], Loss: 0.6901, Accuracy: 0.530
------------------------------------------------------------
Epoch [1/8], Step [500/12500], Loss: 0.6941, Accuracy: 0.355
------------------------------------------------------------
Epoch [1/8], Step [750/12500], Loss: 0.6986, Accuracy: 0.370
------------------------------------------------------------
Epoch [1/8], Step [1000/12500], Loss: 0.6681, Accuracy: 0.700
------------------------------------------------------------
Epoch [1/8], Step [1250/12500], Loss: 0.6953, Accuracy: 0.475
------------------------------------------------------------
Epoch [1/8], Step [1500/12500], Loss: 0.6885, Accuracy: 0.610
------------------------------------------------------------
Epoch [1/8], Step [1750/12500], Loss: 0.6905, Accuracy: 0.477
------------------------------------------------------------
Epoch [1/8], Step [2000/12500], Loss: 0.6889, Accuracy: 0.498
------

In [55]:
model

XORLSTM(
  (lstm): LSTM(1, 2, batch_first=True)
  (fc): Linear(in_features=2, out_features=1, bias=True)
  (activation): Sigmoid()
)

In [57]:
model(XOR(1, 2).generate_data(1)[0]).size()

torch.Size([1, 50, 1])

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=17d2748e-c8ac-4922-b92b-a46154c07520' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>